In [ ]:
def check():
    for i in range(10):
        if i == 2:
            if i % 2 == 0:
                return "even"
            else

In [90]:
import pandas as pd
df = pd.read_csv("/home/ethereum/Downloads/check.csv")

In [91]:
column = []
for i in range(df.shape[0]):
    column.append("False")
print(column)

['False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False', 'False']


In [92]:
df["o_freedelivery"] = column
df.head()

,o_orderkey,o_custkey,o_totalprice,o_orderdate,o_freedelivery
0,23015,548,94748.05,1992-01-12,False
1,38310,548,79316.37,1992-04-15,False
2,17632,548,191202.12,1993-02-22,False
3,34435,548,40953.07,1994-08-06,False
4,35270,548,27725.67,1994-11-10,False


In [93]:
df.iloc[0:1].shape

(1, 5)

In [94]:
def check_freedelivery(total_spent, o_totalprice):
    threshold = 200000
    return (total_spent + float(o_totalprice) >= threshold)

def calc_total(o_custkey, o_orderdate, chain):
    total_spent = 0
    for i in range(chain.shape[0]-2,-1,-1):
        tx_date = chain.iloc[i, 3]
        spent_time = int(o_orderdate[:4]) - int(tx_date[:4]) + int(o_orderdate[5:7])/12 - int(tx_date[5:7])/12 + int(o_orderdate[8:])/365 - int(tx_date[8:])/365
        tx_price = round(chain.iloc[i, 2])
        tx_custkey = chain.iloc[i, 1]
        tx_freedelivery = chain.iloc[i, 4]
            
        if spent_time <= 1:
            if o_custkey == tx_custkey:
                if tx_freedelivery == "False":
                    total_spent += tx_price
                else:
                    return total_spent
        else:
            return total_spent
    return total_spent

In [95]:
total = []
rewards = []
for i in range(df.shape[0]):
    total_spent = calc_total(df.iloc[i, 1], df.iloc[i, 3], df.iloc[:i])
    print(total_spent, end= "=>")
    rewards.append(check_freedelivery(total_spent, df.iloc[i, 2]))
    print(check_freedelivery(total_spent, df.iloc[i, 2]))

0=>False
94748=>False
79316=>True
0=>False
40953=>False
0=>False
186621=>True
254101=>True
546780=>True
0=>False


In [96]:
df["o_freedelivery"] = rewards

In [97]:
print(total)

[]


In [98]:
df.to_csv("newcheck.csv")